# Chapter 11: Advanced OOP Concepts

This chapter explores advanced object-oriented programming concepts in SystemVerilog that are crucial for building sophisticated verification environments and reusable components.

## Parameterized Classes

Parameterized classes allow you to create generic, reusable class templates that can be customized with different data types and parameters.

### Basic Parameterized Classes

```systemverilog
// Generic FIFO class with parameterized width and depth
class generic_fifo #(parameter WIDTH = 8, parameter DEPTH = 16);
    typedef logic [WIDTH-1:0] data_t;
    
    local data_t queue[$];
    local int max_depth;
    
    function new();
        max_depth = DEPTH;
    endfunction
    
    function void push(data_t data);
        if (queue.size() >= max_depth) begin
            $error("FIFO overflow");
            return;
        end
        queue.push_back(data);
    endfunction
    
    function data_t pop();
        if (queue.size() == 0) begin
            $error("FIFO underflow");
            return 0;
        end
        return queue.pop_front();
    endfunction
    
    function int size();
        return queue.size();
    endfunction
    
    function bit is_full();
        return (queue.size() >= max_depth);
    endfunction
    
    function bit is_empty();
        return (queue.size() == 0);
    endfunction
endclass

// Usage examples
module test_parameterized_classes;
    // 8-bit wide, 16 deep FIFO
    generic_fifo #(.WIDTH(8), .DEPTH(16)) byte_fifo;
    
    // 32-bit wide, 64 deep FIFO
    generic_fifo #(.WIDTH(32), .DEPTH(64)) word_fifo;
    
    // 16-bit wide, default depth FIFO
    generic_fifo #(.WIDTH(16)) half_word_fifo;
    
    initial begin
        byte_fifo = new();
        word_fifo = new();
        half_word_fifo = new();
        
        // Test byte FIFO
        byte_fifo.push(8'hAA);
        byte_fifo.push(8'h55);
        $display("Byte FIFO size: %0d", byte_fifo.size());
        $display("Popped: 0x%02h", byte_fifo.pop());
        
        // Test word FIFO
        word_fifo.push(32'hDEADBEEF);
        $display("Word FIFO size: %0d", word_fifo.size());
        $display("Popped: 0x%08h", word_fifo.pop());
    end
endmodule
```

### Advanced Parameterized Classes with Type Parameters

```systemverilog
// Generic container class with type parameter
class generic_container #(type T = int);
    T items[];
    int count;
    
    function new(int initial_size = 10);
        items = new[initial_size];
        count = 0;
    endfunction
    
    function void add(T item);
        if (count >= items.size()) begin
            // Resize array
            T temp[] = new[items.size() * 2];
            temp[0:items.size()-1] = items;
            items = temp;
        end
        items[count] = item;
        count++;
    endfunction
    
    function T get(int index);
        if (index >= 0 && index < count)
            return items[index];
        else begin
            $error("Index %0d out of bounds", index);
            return items[0]; // Return default
        end
    endfunction
    
    function int size();
        return count;
    endfunction
    
    function void display();
        $display("Container contents (%0d items):", count);
        for (int i = 0; i < count; i++) begin
            $display("  [%0d]: %p", i, items[i]);
        end
    endfunction
endclass

// Custom data type for testing
typedef struct {
    string name;
    int age;
    real salary;
} person_t;

module test_type_parameterized;
    // Integer container
    generic_container #(int) int_container;
    
    // String container
    generic_container #(string) string_container;
    
    // Custom type container
    generic_container #(person_t) person_container;
    
    initial begin
        int_container = new(5);
        string_container = new(3);
        person_container = new(2);
        
        // Test integer container
        int_container.add(10);
        int_container.add(20);
        int_container.add(30);
        int_container.display();
        
        // Test string container
        string_container.add("Hello");
        string_container.add("World");
        string_container.display();
        
        // Test custom type container
        person_t p1 = '{"Alice", 30, 50000.0};
        person_t p2 = '{"Bob", 25, 45000.0};
        person_container.add(p1);
        person_container.add(p2);
        person_container.display();
    end
endmodule
```

## Nested Classes

Nested classes allow you to define classes within other classes, providing better encapsulation and organization.

### Basic Nested Classes

```systemverilog
class outer_class;
    int outer_data;
    
    // Nested class definition
    class inner_class;
        int inner_data;
        
        function new(int data);
            inner_data = data;
        endfunction
        
        function void display();
            $display("Inner class data: %0d", inner_data);
        endfunction
    endclass
    
    inner_class inner_obj;
    
    function new(int outer_val, int inner_val);
        outer_data = outer_val;
        inner_obj = new(inner_val);
    endfunction
    
    function void display();
        $display("Outer class data: %0d", outer_data);
        inner_obj.display();
    endfunction
endclass

module test_nested_classes;
    outer_class obj;
    
    initial begin
        obj = new(100, 200);
        obj.display();
    end
endmodule
```

### Complex Nested Class Example - Packet with Headers

```systemverilog
class ethernet_packet;
    // Ethernet header nested class
    class eth_header;
        bit [47:0] dest_mac;
        bit [47:0] src_mac;
        bit [15:0] ether_type;
        
        function new();
            dest_mac = $random();
            src_mac = $random();
            ether_type = 16'h0800; // IP
        endfunction
        
        function void randomize_header();
            dest_mac = $random();
            src_mac = $random();
        endfunction
        
        function void display();
            $display("Ethernet Header:");
            $display("  Dest MAC: %012h", dest_mac);
            $display("  Src MAC:  %012h", src_mac);
            $display("  Type:     %04h", ether_type);
        endfunction
    endclass
    
    // IP header nested class
    class ip_header;
        bit [3:0]  version;
        bit [3:0]  header_length;
        bit [7:0]  tos;
        bit [15:0] total_length;
        bit [15:0] identification;
        bit [2:0]  flags;
        bit [12:0] fragment_offset;
        bit [7:0]  ttl;
        bit [7:0]  protocol;
        bit [15:0] checksum;
        bit [31:0] src_ip;
        bit [31:0] dest_ip;
        
        function new();
            version = 4;
            header_length = 5;
            tos = 0;
            ttl = 64;
            protocol = 8'h06; // TCP
            src_ip = $random();
            dest_ip = $random();
        endfunction
        
        function void display();
            $display("IP Header:");
            $display("  Version: %0d", version);
            $display("  Src IP:  %0d.%0d.%0d.%0d", 
                    src_ip[31:24], src_ip[23:16], src_ip[15:8], src_ip[7:0]);
            $display("  Dest IP: %0d.%0d.%0d.%0d", 
                    dest_ip[31:24], dest_ip[23:16], dest_ip[15:8], dest_ip[7:0]);
            $display("  Protocol: %02h", protocol);
        endfunction
    endclass
    
    eth_header eth_hdr;
    ip_header ip_hdr;
    bit [7:0] payload[];
    
    function new(int payload_size = 64);
        eth_hdr = new();
        ip_hdr = new(); 
        payload = new[payload_size];
        
        // Initialize payload with random data
        foreach(payload[i]) begin
            payload[i] = $random();
        end
    endfunction
    
    function void display();
        $display("=== Ethernet Packet ===");
        eth_hdr.display();
        ip_hdr.display();
        $display("Payload size: %0d bytes", payload.size());
    endfunction
    
    function void randomize_packet();
        eth_hdr.randomize_header();
        ip_hdr.src_ip = $random();
        ip_hdr.dest_ip = $random();
    endfunction
endclass
```

## Copy Constructors

Copy constructors provide a way to create copies of objects, which is essential for proper object management in verification environments.

### Basic Copy Constructor Implementation

```systemverilog
class transaction;
    int id;
    string name;
    bit [31:0] data[];
    real timestamp;
    
    function new(int _id = 0, string _name = "default");
        id = _id;
        name = _name;
        timestamp = $realtime;
        data = new[8]; // Default size
        
        // Initialize with random data
        foreach(data[i]) begin
            data[i] = $random();
        end
    endfunction
    
    // Copy constructor
    function new_copy(transaction original);
        if (original == null) begin
            $error("Cannot copy null transaction");
            return;
        end
        
        this.id = original.id;
        this.name = original.name;
        this.timestamp = $realtime; // New timestamp
        
        // Deep copy of dynamic array
        this.data = new[original.data.size()];
        this.data = original.data; // Copy contents
    endfunction
    
    // Alternative copy method
    function transaction copy();
        transaction new_trans = new();
        new_trans.new_copy(this);
        return new_trans;
    endfunction
    
    function void display();
        $display("Transaction ID: %0d, Name: %s", id, name);
        $display("Timestamp: %0t", timestamp);
        $display("Data size: %0d", data.size());
        $write("Data: ");
        foreach(data[i]) begin
            $write("%08h ", data[i]);
        end
        $display("");
    endfunction
    
    function void modify_data(int index, bit [31:0] value);
        if (index >= 0 && index < data.size()) begin
            data[index] = value;
        end
    endfunction
endclass

module test_copy_constructor;
    transaction original, copy1, copy2;
    
    initial begin
        // Create original transaction
        original = new(1, "original_trans");
        $display("=== Original Transaction ===");
        original.display();
        
        // Create copy using copy constructor
        copy1 = new();
        copy1.new_copy(original);
        $display("\n=== Copy 1 (using copy constructor) ===");
        copy1.display();
        
        // Create copy using copy method
        copy2 = original.copy();
        $display("\n=== Copy 2 (using copy method) ===");
        copy2.display();
        
        // Modify original and show independence
        original.modify_data(0, 32'hDEADBEEF);
        original.name = "modified_original";
        
        $display("\n=== After modifying original ===");
        $display("Original:");
        original.display();
        $display("Copy 1 (should be unchanged):");
        copy1.display();
        $display("Copy 2 (should be unchanged):");
        copy2.display();
    end
endmodule
```

## Shallow vs. Deep Copy

Understanding the difference between shallow and deep copying is crucial for proper object management.

### Demonstrating Shallow vs. Deep Copy

```systemverilog
class inner_data;
    int value;
    string tag;
    
    function new(int v = 0, string t = "default");
        value = v;
        tag = t;
    endfunction
    
    function void display();
        $display("    Inner data - Value: %0d, Tag: %s", value, tag);
    endfunction
    
    function inner_data copy();
        inner_data new_inner = new(this.value, this.tag);
        return new_inner;
    endfunction
endclass

class container;
    int id;
    inner_data inner_obj;
    int simple_array[];
    
    function new(int _id = 0);
        id = _id;
        inner_obj = new(100, "inner");
        simple_array = new[5];
        
        foreach(simple_array[i]) begin
            simple_array[i] = i * 10;
        end
    endfunction
    
    // Shallow copy - copies references, not objects
    function container shallow_copy();
        container new_container = new();
        new_container.id = this.id;
        new_container.inner_obj = this.inner_obj; // Same reference!
        new_container.simple_array = this.simple_array; // Same reference!
        return new_container;
    endfunction
    
    // Deep copy - creates new objects
    function container deep_copy();
        container new_container = new();
        new_container.id = this.id;
        
        // Create new inner object
        new_container.inner_obj = this.inner_obj.copy();
        
        // Create new array and copy contents
        new_container.simple_array = new[this.simple_array.size()];
        new_container.simple_array = this.simple_array;
        
        return new_container;
    endfunction
    
    function void display();
        $display("Container ID: %0d", id);
        inner_obj.display();
        $write("  Array: ");
        foreach(simple_array[i]) begin
            $write("%0d ", simple_array[i]);
        end
        $display("");
    endfunction
    
    function void modify_inner(int new_value, string new_tag);
        inner_obj.value = new_value;
        inner_obj.tag = new_tag;
    endfunction
    
    function void modify_array(int index, int value);
        if (index >= 0 && index < simple_array.size()) begin
            simple_array[index] = value;
        end
    endfunction
endclass

module test_shallow_vs_deep_copy;
    container original, shallow_copy, deep_copy;
    
    initial begin
        // Create original
        original = new(1);
        $display("=== Original Container ===");
        original.display();
        
        // Create shallow copy
        shallow_copy = original.shallow_copy();
        $display("\n=== Shallow Copy ===");
        shallow_copy.display();
        
        // Create deep copy
        deep_copy = original.deep_copy();
        $display("\n=== Deep Copy ===");
        deep_copy.display();
        
        $display("\n=== Modifying Original ===");
        original.modify_inner(999, "modified");
        original.modify_array(0, 555);
        
        $display("Original after modification:");
        original.display();
        
        $display("Shallow copy (affected by modification):");
        shallow_copy.display();
        
        $display("Deep copy (unaffected by modification):");
        deep_copy.display();
        
        // Demonstrate the problem with shallow copy
        $display("\n=== The Shallow Copy Problem ===");
        $display("Shallow copy shares the same inner object and array!");
        $display("Original inner_obj handle: %p", original.inner_obj);
        $display("Shallow copy inner_obj handle: %p", shallow_copy.inner_obj);
        $display("Deep copy inner_obj handle: %p", deep_copy.inner_obj);
        
        $display("Handles are %s", 
                (original.inner_obj == shallow_copy.inner_obj) ? "SAME" : "DIFFERENT");
    end
endmodule
```

## Class Handles and References

Understanding class handles and references is fundamental to working with objects in SystemVerilog.

### Class Handle Basics

```systemverilog
class simple_class;
    int data;
    string name;
    
    function new(int d = 0, string n = "default");
        data = d;
        name = n;
    endfunction
    
    function void display();
        $display("Object: %s, Data: %0d, Handle: %p", name, data, this);
    endfunction
endclass

module test_class_handles;
    simple_class obj1, obj2, obj3;
    
    initial begin
        $display("=== Class Handle Demonstration ===");
        
        // Initially all handles are null
        $display("Initial handle values:");
        $display("obj1: %p", obj1);
        $display("obj2: %p", obj2);
        $display("obj3: %p", obj3);
        
        // Create objects
        obj1 = new(10, "first");
        obj2 = new(20, "second");
        
        $display("\nAfter creating objects:");
        obj1.display();
        obj2.display();
        
        // Handle assignment (not copying the object!)
        obj3 = obj1;
        $display("\nAfter obj3 = obj1:");
        obj1.display();
        obj3.display();
        
        // Modifying through one handle affects the other
        obj3.data = 999;
        obj3.name = "modified";
        
        $display("\nAfter modifying through obj3:");
        obj1.display();
        obj3.display();
        
        // Handle comparison
        $display("\nHandle comparisons:");
        $display("obj1 == obj2: %b", obj1 == obj2);
        $display("obj1 == obj3: %b", obj1 == obj3);
        $display("obj1 === obj3: %b", obj1 === obj3);
        
        // Setting handle to null
        obj1 = null;
        $display("\nAfter obj1 = null:");
        $display("obj1: %p", obj1);
        $display("obj3: %p", obj3);
        obj3.display(); // Still works because object exists
    end
endmodule
```

### Advanced Handle Management

```systemverilog
class managed_object;
    static int object_count = 0;
    int object_id;
    string name;
    
    function new(string n = "unnamed");
        object_count++;
        object_id = object_count;
        name = n;
        $display("Created object %0d: %s", object_id, name);
    endfunction
    
    function void display();
        $display("Object %0d (%s) - Handle: %p", object_id, name, this);
    endfunction
    
    static function int get_object_count();
        return object_count;
    endfunction
endclass

class handle_manager;
    managed_object objects[];
    int count;
    
    function new(int max_objects = 10);
        objects = new[max_objects];
        count = 0;
    endfunction
    
    function int add_object(managed_object obj);
        if (count >= objects.size()) begin
            $display("Handle manager full!");
            return -1;
        end
        
        objects[count] = obj;
        count++;
        return count - 1;
    endfunction
    
    function managed_object get_object(int index);
        if (index >= 0 && index < count) begin
            return objects[index];
        end
        return null;
    endfunction
    
    function void remove_object(int index);
        if (index >= 0 && index < count) begin
            // Shift remaining objects
            for (int i = index; i < count - 1; i++) begin
                objects[i] = objects[i + 1];
            end
            objects[count - 1] = null;
            count--;
        end
    endfunction
    
    function void display_all();
        $display("Handle Manager contains %0d objects:", count);
        for (int i = 0; i < count; i++) begin
            if (objects[i] != null) begin
                $write("  [%0d]: ", i);
                objects[i].display();
            end
        end
    endfunction
    
    function void cleanup();
        for (int i = 0; i < objects.size(); i++) begin
            objects[i] = null;
        end
        count = 0;
        $display("Handle manager cleaned up");
    endfunction
endclass

module test_handle_management;
    handle_manager manager;
    managed_object obj1, obj2, obj3, obj4;
    
    initial begin
        $display("=== Handle Management Test ===");
        
        manager = new(5);
        
        // Create and add objects
        obj1 = new("Alpha");
        obj2 = new("Beta");
        obj3 = new("Gamma");
        
        manager.add_object(obj1);
        manager.add_object(obj2);
        manager.add_object(obj3);
        
        $display("\nTotal objects created: %0d", 
                managed_object::get_object_count());
        
        manager.display_all();
        
        // Get object by index
        obj4 = manager.get_object(1);
        if (obj4 != null) begin
            $display("\nRetrieved object at index 1:");
            obj4.display();
        end
        
        // Remove object
        $display("\nRemoving object at index 1:");
        manager.remove_object(1);
        manager.display_all();
        
        // Cleanup
        $display("\nCleaning up manager:");
        manager.cleanup();
        manager.display_all();
        
        $display("\nFinal object count: %0d", 
                managed_object::get_object_count());
    end
endmodule
```

### Reference Counting and Smart Pointers Pattern

```systemverilog
class ref_counted_object;
    static int total_objects = 0;
    int object_id;
    string data;
    int ref_count;
    
    function new(string d = "default");
        total_objects++;
        object_id = total_objects;
        data = d;
        ref_count = 1; // Creator gets first reference
        $display("Created object %0d with data '%s'", object_id, data);
    endfunction
    
    function void add_ref();
        ref_count++;
        $display("Object %0d ref count increased to %0d", object_id, ref_count);
    endfunction
    
    function void release_ref();
        ref_count--;
        $display("Object %0d ref count decreased to %0d", object_id, ref_count);
        if (ref_count <= 0) begin
            $display("Object %0d being destroyed", object_id);
            // In real implementation, this would trigger cleanup
        end
    endfunction
    
    function int get_ref_count();
        return ref_count;
    endfunction
    
    function void display();
        $display("Object %0d: '%s' (refs: %0d)", object_id, data, ref_count);
    endfunction
endclass

class smart_pointer;
    ref_counted_object obj;
    
    function new(ref_counted_object o = null);
        if (o != null) begin
            obj = o;
            obj.add_ref();
        end
    endfunction
    
    function void assign(ref_counted_object o);
        // Release current object
        if (obj != null) begin
            obj.release_ref();
        end
        
        // Assign new object
        obj = o;
        if (obj != null) begin
            obj.add_ref();
        end
    endfunction
    
    function ref_counted_object get();
        return obj;
    endfunction
    
    function void reset();
        if (obj != null) begin
            obj.release_ref();
            obj = null;
        end
    endfunction
    
    // Destructor simulation
    function void cleanup();
        reset();
    endfunction
endclass

module test_reference_counting;
    ref_counted_object obj1;
    smart_pointer ptr1, ptr2, ptr3;
    
    initial begin
        $display("=== Reference Counting Test ===");
        
        // Create object
        obj1 = new("Test Data");
        obj1.display();
        
        // Create smart pointers
        ptr1 = new(obj1);
        ptr2 = new();
        ptr3 = new();
        
        obj1.display();
        
        // Assign to other pointers
        ptr2.assign(obj1);
        ptr3.assign(obj1);
        
        obj1.display();
        
        // Release one pointer
        $display("\nReleasing ptr1:");
        ptr1.reset();
        obj1.display();
        
        // Release another pointer
        $display("\nReleasing ptr2:");
        ptr2.cleanup();
        obj1.display();
        
        // Release last pointer
        $display("\nReleasing ptr3:");
        ptr3.cleanup();
        obj1.display();
        
        // Release original reference
        $display("\nReleasing original reference:");
        obj1.release_ref();
    end
endmodule
```

## Best Practices and Common Patterns

### Factory Pattern with Parameterized Classes

```systemverilog
// Base transaction class
virtual class base_transaction;
    int id;
    real timestamp;
    
    function new(int _id = 0);
        id = _id;
        timestamp = $realtime;
    endfunction
    
    pure virtual function void display();
    pure virtual function base_transaction copy();
endclass

// Specific transaction types
class read_transaction extends base_transaction;
    bit [31:0] address;
    
    function new(int _id = 0, bit [31:0] addr = 0);
        super.new(_id);
        address = addr;
    endfunction
    
    virtual function void display();
        $display("Read Transaction - ID: %0d, Addr: 0x%08h, Time: %0t", 
                id, address, timestamp);
    endfunction
    
    virtual function base_transaction copy();
        read_transaction new_trans = new(this.id, this.address);
        return new_trans;
    endfunction
endclass

class write_transaction extends base_transaction;
    bit [31:0] address;
    bit [31:0] data;
    
    function new(int _id = 0, bit [31:0] addr = 0, bit [31:0] d = 0);
        super.new(_id);
        address = addr;
        data = d;
    endfunction
    
    virtual function void display();
        $display("Write Transaction - ID: %0d, Addr: 0x%08h, Data: 0x%08h, Time: %0t", 
                id, address, data, timestamp);
    endfunction
    
    virtual function base_transaction copy();
        write_transaction new_trans = new(this.id, this.address, this.data);
        return new_trans;
    endfunction
endclass

// Transaction factory
class transaction_factory #(type T = base_transaction);
    static int next_id = 1;
    
    static function T create();
        T trans = new(next_id++);
        return trans;
    endfunction
    
    static function T create_with_id(int id);
        T trans = new(id);
        return trans;
    endfunction
endclass

module test_factory_pattern;
    // Specialized factories
    transaction_factory #(read_transaction) read_factory;
    transaction_factory #(write_transaction) write_factory;
    
    read_transaction rd_trans;
    write_transaction wr_trans;
    base_transaction trans_array[];
    
    initial begin
        $display("=== Factory Pattern Test ===");
        
        trans_array = new[4];
        
        // Create transactions using factories
        rd_trans = read_factory::create();
        rd_trans.address = 32'h1000;
        trans_array[0] = rd_trans;
        
        wr_trans = write_factory::create();
        wr_trans.address = 32'h2000;
        wr_trans.data = 32'hDEADBEEF;
        trans_array[1] = wr_trans;
        
        rd_trans = read_factory::create();
        rd_trans.address = 32'h3000;
        trans_array[2] = rd_trans;
        
        wr_trans = write_factory::create();
        wr_trans.address = 32'h4000;
        wr_trans.data = 32'hCAFEBABE;
        trans_array[3] = wr_trans;
        
        // Display all transactions
        foreach(trans_array[i]) begin
            trans_array[i].display();
        end
        
        // Test copying
        $display("\n=== Testing Copy ===");
        base_transaction copied = trans_array[1].copy();
        copied.display();
    end
endmodule
```

# SystemVerilog Advanced OOP - Simple Examples by Topic

## Parameterized Classes

### 1. Simple Generic Stack
**Description:** Basic Stack class with parameterized depth and data width. Shows push(), pop(), and size() operations with different configurations.

### 2. Generic Counter
**Description:** Counter class with parameterized width and increment value. Demonstrates customizable counting behavior with overflow detection.

### 3. Configurable Buffer
**Description:** Simple Buffer class with parameterized size and data type. Shows basic read/write operations with different buffer sizes.

### 4. Generic Register Bank
**Description:** Register bank with parameterized number of registers and register width. Basic read/write functionality for different configurations.

### 5. Parameterized Memory Model
**Description:** Simple memory model with configurable address width and data width. Basic memory read/write operations.

## Advanced Parameterized Classes with Type Parameters

### 6. Generic Array Container
**Description:** Container class that can hold any data type (int, string, custom structs). Shows add(), get(), and display() methods.

### 7. Type-Safe Queue
**Description:** Queue implementation that works with different data types. Demonstrates enqueue/dequeue operations with type safety.

### 8. Generic Pair Class
**Description:** Simple class that holds two values of potentially different types. Shows parameterization with multiple type parameters.

### 9. Flexible Scoreboard
**Description:** Scoreboard class that can compare any data type. Demonstrates generic comparison and tracking functionality.

### 10. Universal Cache
**Description:** Simple cache implementation that works with any key-value pair types. Basic get/put operations with different data types.

## Nested Classes

### 11. Simple Car with Engine
**Description:** Car class with nested Engine class. Shows basic composition with engine start/stop functionality inside car object.

### 12. Book with Chapter
**Description:** Book class containing nested Chapter class. Demonstrates hierarchical organization of related data structures.

### 13. Computer with Components
**Description:** Computer class with nested CPU, RAM, and Storage classes. Shows complex object composition with nested classes.

### 14. Bank Account with Transaction
**Description:** BankAccount class with nested Transaction class for tracking deposits and withdrawals internally.

### 15. Simple HTTP Request
**Description:** HTTPRequest class with nested Header and Body classes. Shows network protocol modeling with nested structures.

## Copy Constructors

### 16. Student Record Copy
**Description:** Student class with copy constructor for creating identical student records. Shows proper copying of all student data.

### 17. Simple Message Copy
**Description:** Message class demonstrating copy constructor for duplicating messages with timestamps and content.

### 18. Configuration Copy
**Description:** Config class with copy constructor for duplicating system configurations safely.

### 19. Basic Transaction Copy
**Description:** Transaction class showing how to properly copy transaction data including IDs and amounts.

### 20. Simple Data Packet Copy
**Description:** DataPacket class with copy constructor for network packet duplication with header and payload copying.

## Shallow vs. Deep Copy

### 21. Address Book Example
**Description:** Person class with Address object showing difference between shallow copy (shared address) and deep copy (separate address).

### 22. Team with Players
**Description:** Team class containing Player objects. Demonstrates shallow vs deep copying of team rosters.

### 23. Shopping Cart Demo
**Description:** ShoppingCart with Item objects showing how shallow copy shares items while deep copy creates separate item lists.

### 24. File System Node
**Description:** FileNode with child nodes showing tree structure copying. Shallow copy shares subtrees, deep copy duplicates entire tree.

### 25. Simple Database Record
**Description:** DatabaseRecord with linked data showing copying implications for related records.

## Class Handles and References

### 26. Basic Handle Demo
**Description:** Simple class showing handle assignment vs object copying. Demonstrates that handles point to same object.

### 27. Handle Array Management
**Description:** Array of class handles showing how multiple handles can reference the same objects.

### 28. Null Handle Safety
**Description:** Demonstrates null handle checking and safe object access patterns.

### 29. Handle Comparison
**Description:** Shows difference between handle equality (==) and object content comparison.

### 30. Simple Object Pool
**Description:** Basic object pool implementation managing reusable objects through handles.

## Advanced Handle Management

### 31. Reference Counter
**Description:** Simple class with reference counting to track how many handles point to an object.

### 32. Object Registry
**Description:** Central registry managing all objects by handles with lookup and cleanup functionality.

### 33. Handle Lifecycle Demo
**Description:** Shows object creation, handle assignment, and cleanup in controlled sequence.

### 34. Weak Reference Pattern
**Description:** Basic weak reference implementation that doesn't prevent object cleanup.

### 35. Handle Validation
**Description:** Utility functions for validating handles before use and detecting dangling references.

## Reference Counting and Smart Pointers Pattern

### 36. Smart Pointer Basics
**Description:** Simple smart pointer class that automatically manages object lifetime through reference counting.

### 37. Shared Resource Manager
**Description:** Resource class with automatic cleanup when no more references exist.

### 38. Automatic Memory Management
**Description:** Demonstrates automatic object destruction when reference count reaches zero.

### 39. Circular Reference Detection
**Description:** Simple example showing circular reference problem and basic detection methods.

### 40. Resource Pool with Counting
**Description:** Object pool using reference counting to manage resource allocation and deallocation.

## Best Practices and Design Patterns

### 41. Simple Factory with Types
**Description:** Factory class creating different object types based on string or enum parameters.

### 42. Template Method Pattern
**Description:** Base class with template method and derived classes implementing specific steps.

### 43. Basic Builder Pattern
**Description:** Builder class for constructing complex objects step by step with method chaining.

### 44. Simple Observer Pattern
**Description:** Subject class notifying multiple observer objects of state changes.

### 45. Strategy Pattern with Generics
**Description:** Generic strategy pattern allowing different algorithms to be plugged in at runtime.

## Common Pitfalls and Debugging

### 46. Parameter vs Type Confusion
**Description:** Side-by-side examples showing parameter vs type parameterization differences.

### 47. Handle Leak Detection
**Description:** Simple techniques for detecting when objects aren't being properly cleaned up.

### 48. Copy Constructor Pitfalls
**Description:** Common mistakes in copy constructors and their correct implementations.

### 49. Nested Class Access
**Description:** Examples of correct ways to access nested class members and common access errors.

### 50. Generic Type Safety
**Description:** Demonstrates type safety benefits and common type-related errors in parameterized classes.